In [1]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import time

In [2]:
url = 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do?action=page&searchCriteria.page=1'

In [3]:
url_for_blocks = 'https://citydev-portal.edinburgh.gov.uk'

In [4]:
with open('House.csv','w', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter = ",")
    writer.writerow(
        [
            'link_url',
            'name_link',
            'adress',
            'link',
            'validated',
            'status',
            'document_type',
            'document_link'
        ])

In [9]:
count = 1
for j in range(1,13):
    cookies = {
        'JSESSIONID': 'b9yJIaJS7snXtjl6hPnfYDr8ZVuapamL_-vfe9ll.lap-pubacc2',
    }

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        # 'Cookie': 'JSESSIONID=b9yJIaJS7snXtjl6hPnfYDr8ZVuapamL_-vfe9ll.lap-pubacc2',
        'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do?action=page&searchCriteria.page=2',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'action': 'page',
        'searchCriteria.page': '1',
    }

    response = requests.get(
        'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do',
        params=params,
        cookies=cookies,
        headers=headers,
    )

    src = response.text

    with open(f"index{j}.html", "w", encoding="utf-8") as file:
        file.write(src) #important, or can banned

    with open(f'index{j}.html', encoding="utf-8") as file:
        src = file.read()

    soup = BeautifulSoup(src, 'lxml')

    all_list_data = soup.find_all(class_='searchresult')

    for i in range(len(all_list_data)):
        link_url = url_for_blocks+all_list_data[i].find('a').get('href')
        name_link = all_list_data[i].find('a').text.strip()
        adress = all_list_data[i].find(class_='address').text.strip()

        meta_info = all_list_data[i].find(class_='metaInfo').text.strip()

        link = meta_info[meta_info.find('Ref. No:')+9:meta_info.find('Ref. No:')+50].strip()
        validated = meta_info[meta_info.find('Validated:')+10:meta_info.find('Validated:')+60].strip()
        status = meta_info[meta_info.find('Status:')+9:meta_info.find('Status:')+90].strip()

        personal = link_url[link_url.find('=')+1:link_url.find('&')]
        personal_link_adress = 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?activeTab=documents&keyVal='+personal



        cookies = {
            'JSESSIONID': 'b9yJIaJS7snXtjl6hPnfYDr8ZVuapamL_-vfe9ll.lap-pubacc2',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
            'Connection': 'keep-alive',
            # 'Cookie': 'JSESSIONID=b9yJIaJS7snXtjl6hPnfYDr8ZVuapamL_-vfe9ll.lap-pubacc2',
            'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?keyVal=RRICZNEWGRP00&activeTab=summary',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
        }

        params = {
            'activeTab': 'documents',
            'keyVal': f'{personal}',
        }

        rsc = requests.get(
            'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do',
            params=params,
            cookies=cookies,
            headers=headers,
        )
        src = rsc.text
        with open(f"{personal}_{count}.html", "w", encoding="utf-8") as file:
            file.write(src) #important, or can banned
        with open(f"{personal}_{count}.html", encoding="utf-8") as file:
            src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        print(count)
        count+=1
        documents_data = soup.find('table', id='Documents').find_all('tr')
        for i in range(1,len(documents_data)):
            document_data = documents_data[i].find_all('td')
            document_type = document_data[2].text
            document_link = url_for_blocks+document_data[5].find('a').get('href')
            with open('House.csv','a', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(
                        [
                            link_url,
                            name_link,
                            adress,
                            link,
                            validated,
                            status,
                            document_type,
                            document_link
                        ])
        time.sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


AttributeError: 'NoneType' object has no attribute 'find_all'

In [6]:
for j in range(1,8):
    cookies = {
        'JSESSIONID': 'pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
    }

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
        'Connection': 'keep-alive',
        # 'Cookie': 'JSESSIONID=pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
        'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do?action=page&searchCriteria.page=2',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'action': 'page',
        'searchCriteria.page': f'{j}',
    }

    response = requests.get(
        'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do',
        params=params,
        cookies=cookies,
        headers=headers,
    )

    src = response.text

    with open(f"index{j}.html", "w", encoding="utf-8") as file:
        file.write(src) #important, or can banned

    with open(f'index{j}.html', encoding="utf-8") as file:
        src = file.read()

    soup = BeautifulSoup(src, 'lxml')

    all_list_data = soup.find_all(class_='searchresult')

    for i in range(len(all_list_data)):
        link_url = url_for_blocks+all_list_data[i].find('a').get('href')
        name_link = all_list_data[i].find('a').text.strip()
        adress = all_list_data[i].find(class_='address').text.strip()

        meta_info = all_list_data[i].find(class_='metaInfo').text.strip()

        link = meta_info[meta_info.find('Ref. No:')+9:meta_info.find('Ref. No:')+50].strip()
        validated = meta_info[meta_info.find('Validated:')+10:meta_info.find('Validated:')+60].strip()
        status = meta_info[meta_info.find('Status:')+9:meta_info.find('Status:')+90].strip()

        personal = link_url[link_url.find('=')+1:link_url.find('&')]
        personal_link_adress = 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?activeTab=documents&keyVal='+personal



        cookies = {
            'JSESSIONID': 'pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            # 'Cookie': 'JSESSIONID=pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
            'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?keyVal=RRLUDHEWH8O00&activeTab=summary',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
        }

        params = {
            'activeTab': 'documents',
            'keyVal': f'{personal}',
        }

        response = requests.get(
            'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do',
            params=params,
            cookies=cookies,
            headers=headers,
        )
        src = rsc.text
        with open(f"{count}.html", "w", encoding="utf-8") as file:
            file.write(src) #important, or can banned
        with open(f"{count}.html", encoding="utf-8") as file:
            src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        print(count)
        count+=1
        documents_data = soup.find('table', id='Documents').find_all('tr')
        for i in range(1,len(documents_data)):
            document_data = documents_data[i].find_all('td')
            document_type = document_data[2].text
            document_link = url_for_blocks+document_data[5].find('a').get('href')
            with open('House.csv','a', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(
                        [
                            link_url,
                            name_link,
                            adress,
                            link,
                            validated,
                            status,
                            document_type,
                            document_link
                        ])
            

In [ ]:
for j in range(1,59):
    cookies = {
        'JSESSIONID': 'pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
    }

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
        'Connection': 'keep-alive',
        # 'Cookie': 'JSESSIONID=pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
        'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do?action=page&searchCriteria.page=2',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'action': 'page',
        'searchCriteria.page': f'{j}',
    }

    response = requests.get(
        'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do',
        params=params,
        cookies=cookies,
        headers=headers,
    )

    src = response.text

    with open(f"index{j}.html", "w", encoding="utf-8") as file:
        file.write(src) #important, or can banned

    with open(f'index{j}.html', encoding="utf-8") as file:
        src = file.read()

    soup = BeautifulSoup(src, 'lxml')

    all_list_data = soup.find_all(class_='searchresult')

    for i in range(len(all_list_data)):
        link_url = url_for_blocks+all_list_data[i].find('a').get('href')
        name_link = all_list_data[i].find('a').text.strip()
        adress = all_list_data[i].find(class_='address').text.strip()

        meta_info = all_list_data[i].find(class_='metaInfo').text.strip()

        link = meta_info[meta_info.find('Ref. No:')+9:meta_info.find('Ref. No:')+50].strip()
        validated = meta_info[meta_info.find('Validated:')+10:meta_info.find('Validated:')+60].strip()
        status = meta_info[meta_info.find('Status:')+9:meta_info.find('Status:')+90].strip()

        personal = link_url[link_url.find('=')+1:link_url.find('&')]
        personal_link_adress = 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?activeTab=documents&keyVal='+personal



        cookies = {
            'JSESSIONID': 'pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            # 'Cookie': 'JSESSIONID=pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
            'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?keyVal=RRLUDHEWH8O00&activeTab=summary',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
        }

        params = {
            'activeTab': 'documents',
            'keyVal': f'{personal}',
        }

        response = requests.get(
            'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do',
            params=params,
            cookies=cookies,
            headers=headers,
        )
        src = rsc.text
        with open(f"{count}.html", "w", encoding="utf-8") as file:
            file.write(src) #important, or can banned
        with open(f"{count}.html", encoding="utf-8") as file:
            src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        print(count)
        count+=1
        documents_data = soup.find('table', id='Documents').find_all('tr')
        for i in range(1,len(documents_data)):
            document_data = documents_data[i].find_all('td')
            document_type = document_data[2].text
            document_link = url_for_blocks+document_data[5].find('a').get('href')
            with open('House.csv','a', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(
                        [
                            link_url,
                            name_link,
                            adress,
                            link,
                            validated,
                            status,
                            document_type,
                            document_link
                        ])

# 3 teg

In [ ]:
cookies = {
    'JSESSIONID': 'pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
    'Connection': 'keep-alive',
    # 'Cookie': 'JSESSIONID=pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
    'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do?action=page&searchCriteria.page=2',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'action': 'page',
    'searchCriteria.page': f'{i}',
}

response = requests.get(
    'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/pagedSearchResults.do',
    params=params,
    cookies=cookies,
    headers=headers,
)

In [ ]:
cookies = {
    'JSESSIONID': 'pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'JSESSIONID=pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
    'Referer': 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?keyVal=RRLUDHEWH8O00&activeTab=summary',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'activeTab': 'documents',
    'keyVal': f'{personal}',
}

response = requests.get(
    'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do',
    params=params,
    cookies=cookies,
    headers=headers,
)

In [12]:
len(all_list_data)

10

In [127]:
#all_list_data[0]

In [76]:
link_url = url_for_blocks+all_list_data[0].find('a').get('href')

In [74]:
name_link = all_list_data[0].find('a').text.strip()

In [30]:
adress = all_list_data[0].find(class_='address').text.strip()

In [44]:
meta_info = all_list_data[0].find(class_='metaInfo').text.strip()

In [68]:
link = meta_info[meta_info.find('Ref. No:')+9:meta_info.find('Ref. No:')+50].strip()

In [52]:
validated = meta_info[meta_info.find('Validated:')+10:meta_info.find('Validated:')+60].strip()

In [57]:
status = meta_info[meta_info.find('Status:')+9:meta_info.find('Status:')+90].strip()

In [80]:
personal = link_url[link_url.find('=')+1:link_url.find('&')]

In [79]:
link_url

'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?keyVal=RRICZNEWGRP00&activeTab=summary'

In [81]:
personal_link_adress = 'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?activeTab=documents&keyVal='+personal

In [82]:
personal_link_adress

'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do?activeTab=documents&keyVal=RRICZNEWGRP00'

In [93]:
cookies = {
    'JSESSIONID': 'pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'JSESSIONID=pKzuHLEy3Z8pfxT3f_7XrWzlaba2EeNGZrVv95Yr.lap-pubacc2',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'activeTab': 'documents',
    'keyVal': personal,
}

rsc = requests.get(
    'https://citydev-portal.edinburgh.gov.uk/idoxpa-web/applicationDetails.do',
    params=params,
    cookies=cookies,
    headers=headers,
)

In [100]:
src = rsc.text

In [101]:
with open(f"{personal}.html", "w", encoding="utf-8") as file:
    file.write(src) #important, or can banned

In [104]:
with open(f"{personal}.html", encoding="utf-8") as file:
    src = file.read()

In [105]:
soup = BeautifulSoup(src, 'lxml')

In [109]:
documents_data = soup.find('table', id='Documents').find_all('tr')

In [114]:
documents_data = tmp.find_all('tr')

In [125]:
for i in range(1,len(documents_data)):
    document_data = documents_data[i].find_all('td')
    document_type = document_data[2].text
    document_link = url_for_blocks+document_data[5].find('a').get('href')
    with open('House.csv','a', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(
                [
                    link_url,
                    name_link,
                    adress,
                    link,
                    validated,
                    status,
                    document_type,
                    document_link
                ])